In [ ]:
%pip install pyspark findspark

In [1]:
import iceberg_spark
spark, jvm, hive_uri, sc = iceberg_spark.start_spark('s3', 'aws', 'hadoop')

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.iceberg#iceberg-spark3-runtime added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e72489c7-3ad1-4bbb-b783-e04059f8b151;1.0
	confs: [default]


:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.iceberg#iceberg-spark3-runtime;0.12.0 in central
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
:: resolution report :: resolve 153ms :: artifacts dl 4ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.0 from central in [default]
	org.apache.iceberg#iceberg-spark3-runtime;0.12.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-e72489c7-3ad1-4bbb-b783-e04059f8b151
	confs: [default

In [2]:
# Create table
spark.sql("""
CREATE OR REPLACE TABLE sample (
    id bigint,
    data string,
    category string)
USING iceberg
PARTITIONED BY (category)""")

spark.read.table('sample').show()

21/11/18 03:32:58 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+---+----+--------+
| id|data|category|
+---+----+--------+
+---+----+--------+



In [3]:
# insert records into a table
spark.sql("""INSERT INTO sample VALUES (1, 'a', 'orders'), (2, 'b', 'product')""")

spark.read.table('sample').show()

+---+----+--------+
| id|data|category|
+---+----+--------+
|  1|   a|  orders|
|  2|   b| product|
+---+----+--------+



In [ ]:
# update records in a table
spark.sql("""
UPDATE default.sample
SET data = 'updated_data'
WHERE category = 'orders'""")

spark.read.table('sample').show()

In [ ]:
# Using Java compact the table for performance
catalog = jvm.CatalogUtil.loadCatalog("org.apache.iceberg.hive.HiveCatalog", "spark_catalog", {'uri': hive_uri}, sc._jsc.hadoopConfiguration())

# Select the table
table_name = jvm.TableIdentifier.parse("default.sample")
table = catalog.loadTable(table_name)

# Run the compact
jvm.Actions.forTable(table).rewriteDataFiles().targetSizeInBytes(500 * 1024 * 1024).execute()
print("Table compacted")

In [ ]:
# delete records from a table
spark.sql("""DELETE FROM sample WHERE category = 'orders'""")

spark.read.table('sample').show()

In [ ]:
# Droping tables
spark.sql("DROP TABLE sample")

In [ ]:
spark.read.table('sample').show()

# Writing with DataFrames
[apache Iceberg](https://iceberg.apache.org/spark-writes/#writing-with-dataframes)
<ul>
    <li>df.writeTo(t).create() is equivalent to CREATE TABLE AS SELECT</li>
    <li>df.writeTo(t).replace() is equivalent to REPLACE TABLE AS SELECT</li>
    <li>df.writeTo(t).append() is equivalent to INSERT INTO</li>
    <li>df.writeTo(t).overwritePartitions() is equivalent to dynamic INSERT OVERWRITE</li>
</ul>
